In [1]:
import pandas_datareader.data as web
import pandas as pd 
import numpy as np 
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

In [2]:
# Black Scholes Merton Valuation Model

def N(z):
    '''
    Normal cumulative density function which returns the 
    cumulative density under the normal curve along the 
    point 'z' where the cumulative density is calculated.
    Refer to scipy.stats documentation for more information
    '''
    from scipy.stats import norm

    return norm.cdf(z)


def call_value(S, K, r, t, vol):
    '''
    Returns the Black-Scholes call option value where
    the parameters have their usual meanings.

    :param S : Underlying stock price
    :param K : Strike price
    :param r : Risk free rate (Usually treasury bond rates or bank rates)
    :param vol : volatility of the stock
    :param t : time to expiration (T - t in documentation)
    '''
    d1 = (1.0/(vol * np.sqrt(t))) * (np.log(S / K) + t * (r + (vol ** 2.0) / 2))
    d2 = d1 - vol * np.sqrt(t)

    return N(d1) * S - N(d2) * K * np.exp(-r * t), N(-d2), d1

def put_value(S, K, r, t, vol):
    '''
    Returns the Black-Scholes put option value where
    the parameters have their usual meanings.

    :param S : Underlying stock price
    :param K : Strike price
    :param r : Risk free rate (Usually treasury bond rates or bank rates)
    :param vol : volatility of the stock
    :param t : time to expiration (T - t in documentation)
    '''
    d1 = (1.0/(vol * np.sqrt(t))) * (np.log(S / K) + t * (r + (vol ** 2.0) / 2))
    d2 = d1 - vol * np.sqrt(t)

    return  N(-d2) * K * np.exp(-r * t) - N(-d1) * S

The Merton Model is the original structural model, where default can only occur at maturity when the market value of assets equals the total book value of liabilities. At this point it is useful to determine the values of the parameters needed in the estimation of real default probabilities. Our aim is to select empirically reasonable parameter values. Since, the term structure of Expected Default Frequencies (EDFs) produced by the models will be compared with the default probabilities provided by Moody’s for bonds over the corresponding time period of data taken, the chosen parameter values should represent that period.

>$$EDF = N(-\frac{ln({\frac{V_a}{P}}) + (\mu + \sigma_a ^ 2 / 2 )t}{\sigma_a {t^{1/2}}} ) = N(-DD)$$

The value of assets is denoted as Va and the volatility of assets as $\sigma_a$ . P is the face value of the Debt. The real default probability for this model is calculated using the Distance to Default (DD) measure that includes the actual drift µ (equals the asset risk premium plus the risk-free interest rate) instead of the risk-free interest rate. This is crucial, since failure to include the actual drift µ, will result in a riskneutral probability of default.

In [3]:
start = datetime(2000, 1, 1)
end = datetime(2021, 4, 18)
start1 = '2000-01-01'
end1 = '2021-04-18'

infy = pd.DataFrame()
infy['INFY'] = web.DataReader('INFY.NS', 'yahoo', start, end)['Close']

In [4]:
infy.head()

,INFY
Date,
2000-01-03,122.071877
2000-01-04,131.686722
2000-01-05,121.151955
2000-01-06,111.459763
2000-01-07,102.542969


In [5]:
t = 252 
rets = infy.pct_change()[1:].tail(t)

In [6]:
rets.tail()

,INFY
Date,
2021-04-09,0.000833
2021-04-12,-0.010617
2021-04-13,-0.020060
2021-04-15,-0.026053
2021-04-16,-0.005144


In [7]:
mu = rets.mean()
sig = rets.std()

E = 65844  # consolidated shareholder equity
V = 92768 # total assets 
P = V - E # total liabilities

In [8]:
P

26924

In [9]:
val, prob,d1 = call_value(V, P, mu, t, sig)

In [10]:
prob[0] * 100

3.828083940431273e-09

In [11]:
d1

INFY    6.813135
dtype: float64